<a href="https://colab.research.google.com/github/areias/healthcare_fraud/blob/main/Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [6]:
train_ben=pd.read_csv('Train_Beneficiarydata-1542865627584.csv')
train_in=pd.read_csv('Train_Inpatientdata-1542865627584.csv')
train_out=pd.read_csv('Train_Outpatientdata-1542865627584.csv')
train=pd.read_csv('Train-1542865627584.csv')
test_ben=pd.read_csv('Test_Beneficiarydata-1542969243754.csv')
test_in=pd.read_csv('Test_Inpatientdata-1542969243754.csv')
test_out=pd.read_csv('Test_Outpatientdata-1542969243754.csv')
test=pd.read_csv('Test-1542969243754.csv')

**Merging the data**

**Train**

In [7]:
train_in['Claim_type']='Inpatient'
train_out['Claim_type']='Outpatient'

In [8]:
final_train=pd.concat([train_in, train_out])
final_train=pd.merge(train_ben, final_train, on='BeneID')
final_train=pd.merge(final_train, train, on='Provider')

**Test**

In [14]:
test_in['Claim_type']='Inpatient'
test_out['Claim_type']='Outpatient'

In [15]:
final_test=pd.concat([test_in, test_out])
final_test=pd.merge(test_ben, final_test, on='BeneID')
final_test=pd.merge(final_test, test, on='Provider')

In [12]:
final_train.head()

,BeneID,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DischargeDt,DiagnosisGroupCode,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,ClmDiagnosisCode_4,ClmDiagnosisCode_5,ClmDiagnosisCode_6,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,Claim_type,PotentialFraud
0,BENE11001,1943-01-01,NaN,1,1,0,39,230,12,12,1,2,1,2,2,1,1,1,2,1,1,36000,3204,60,70,CLM46614,2009-04-12,2009-04-18,PRV55912,26000,PHY390922,NaN,NaN,2009-04-12,7866,1068.0,2009-04-18,201,1970,4019,5853,7843,2768,71590,2724,19889,5849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Inpatient,Yes
1,BENE12511,1944-09-01,NaN,1,1,0,39,510,12,12,2,2,2,2,2,2,1,1,1,2,2,0,0,1250,90,CLM724430,2009-12-11,2009-12-11,PRV55912,600,PHY420794,NaN,NaN,NaN,NaN,0.0,NaN,NaN,V560,V4283,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Outpatient,Yes
2,BENE14577,1935-10-01,NaN,2,1,Y,33,70,12,12,2,1,1,2,2,2,2,1,2,2,1,0,0,2390,440,CLM567180,2009-09-07,2009-09-07,PRV55912,20,PHY381733,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Outpatient,Yes
3,BENE15074,1932-06-01,NaN,2,1,0,39,470,12,12,1,2,1,2,2,2,1,1,2,2,2,0,0,870,210,CLM259984,2009-03-22,2009-03-22,PRV55912,30,PHY371570,NaN,PHY393406,NaN,NaN,0.0,NaN,NaN,V812,40390,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Outpatient,Yes
4,BENE16973,1932-05-01,NaN,1,1,0,39,310,12,12,1,1,1,2,2,2,2,1,2,2,2,24000,2136,450,200,CLM565430,2009-09-06,2009-09-06,PRV55912,50,PHY365867,PHY327147,NaN,NaN,NaN,0.0,NaN,NaN,V7183,53081,78959,4280,E8788,78079,79902,25002,71848,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Outpatient,Yes


In [11]:
final_train.shape

(558211, 56)

In [9]:
final_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 558211 entries, 0 to 558210
Data columns (total 56 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   BeneID                           558211 non-null  object 
 1   DOB                              558211 non-null  object 
 2   DOD                              4131 non-null    object 
 3   Gender                           558211 non-null  int64  
 4   Race                             558211 non-null  int64  
 5   RenalDiseaseIndicator            558211 non-null  object 
 6   State                            558211 non-null  int64  
 7   County                           558211 non-null  int64  
 8   NoOfMonths_PartACov              558211 non-null  int64  
 9   NoOfMonths_PartBCov              558211 non-null  int64  
 10  ChronicCond_Alzheimer            558211 non-null  int64  
 11  ChronicCond_Heartfailure         558211 non-null  int64  
 12  Ch

In [17]:
final_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 135392 entries, 0 to 135391
Data columns (total 55 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   BeneID                           135392 non-null  object 
 1   DOB                              135392 non-null  object 
 2   DOD                              1040 non-null    object 
 3   Gender                           135392 non-null  int64  
 4   Race                             135392 non-null  int64  
 5   RenalDiseaseIndicator            135392 non-null  object 
 6   State                            135392 non-null  int64  
 7   County                           135392 non-null  int64  
 8   NoOfMonths_PartACov              135392 non-null  int64  
 9   NoOfMonths_PartBCov              135392 non-null  int64  
 10  ChronicCond_Alzheimer            135392 non-null  int64  
 11  ChronicCond_Heartfailure         135392 non-null  int64  
 12  Ch

**Usage of the date variables: **

In [18]:
# Convert the column Date into Datetime64 datatype
final_train['ClaimStartDt'] = pd.to_datetime(final_train['ClaimStartDt'], format = '%Y-%m-%d')
final_train['ClaimEndDt'] = pd.to_datetime(final_train['ClaimEndDt'], format = '%Y-%m-%d')
final_test['ClaimStartDt'] = pd.to_datetime(final_test['ClaimStartDt'], format = '%Y-%m-%d')
final_test['ClaimEndDt'] = pd.to_datetime(final_test['ClaimEndDt'], format = '%Y-%m-%d')